Installing spark dependencies

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xzf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark==3.0.3

--2022-11-16 02:28:14--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-11-16 02:28:14 ERROR 404: Not Found.

tar (child): spark-3.3.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209.1 MB 67 kB/s 
     |████████████████████████████████| 198 kB 59.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435970 sha256=b9d9cfa1054fe31cea431d985c792a1aa07b7d15cb8b1306f583fb641a48e36d
  Stored in directory: /root/.cache/pip/wheels/7e/6d/0a/6b0bf301bc

Import required libraries

In [38]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *

from datetime import datetime
import json

Create a Spark Session

In [4]:
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

Mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Load Dataset into dataframe

In [5]:
# Load the dataset
data = spark.read.load('/content/drive/MyDrive/PBDM_Dataset/Sample-Spreadsheet-500000-rows.csv', format='csv', inferSchema=True, header=True)

# Print schema
data.printSchema()

root
 |-- Eldon Base for stackable storage shelf, platinum: string (nullable = true)
 |-- Muhammed MacIntyre: string (nullable = true)
 |-- 3: string (nullable = true)
 |-- -213.25: string (nullable = true)
 |-- 38.94: string (nullable = true)
 |-- 35: string (nullable = true)
 |-- Nunavut: string (nullable = true)
 |-- Storage & Organization: string (nullable = true)
 |-- 0.8: string (nullable = true)



In [7]:
# The number of rows in the dataset
data.count()

59506

In [6]:
# See first 10 rows of the dataset
data.show(10)

+------------------------------------------------+------------------+---+-------+------+-----+-------+----------------------+----+
|Eldon Base for stackable storage shelf, platinum|Muhammed MacIntyre|  3|-213.25| 38.94|   35|Nunavut|Storage & Organization| 0.8|
+------------------------------------------------+------------------+---+-------+------+-----+-------+----------------------+----+
|                            "1.7 Cubic Foot C...|      Barry French|293| 457.81|208.16|68.02|Nunavut|            Appliances|0.58|
|                            Cardinal Slant-D�...|      Barry French|293|  46.71|  8.69| 2.99|Nunavut|  Binders and Binde...|0.39|
|                                            R380|     Clay Rozendal|483|1198.97|195.99| 3.99|Nunavut|  Telephones and Co...|0.58|
|                            Holmes HEPA Air P...|    Carlos Soltero|515|  30.94| 21.78| 5.94|Nunavut|            Appliances| 0.5|
|                            G.E. Longer-Life ...|    Carlos Soltero|515|   4.43|  

Convert CSV to JSON

In [37]:
start_time = datetime.now()
print ("Reading CSV file started at : ",start_time)
df = spark.read.load('/content/drive/MyDrive/PBDM_Dataset/Sample-Spreadsheet-500000-rows.csv', format='csv', inferSchema=True, header=True)
df.write.mode("overwrite").format("json").save("/content/drive/MyDrive/PBDM_Dataset/json_output")
# df.to_json("output_data.json",sep=',',index=None)
# df.toJSON("/content/drive/MyDrive/PBDM_Dataset/json_output")
# json_output = df.toJSON()
end_time = datetime.now()
print ("Writing to JSON completed at: ",end_time)
print ("Total Duration in Converting CSV to JSON: ",end_time-start_time)

Reading CSV file started at :  2022-11-16 03:39:09.828771
Writing to JSON completed at:  2022-11-16 03:39:11.489498
Total Duration in Converting CSV to JSON:  0:00:01.660727


In [28]:
# for row in json_output.collect():
#     #json string
#     print(row) 

#     print("###################################################################################################################")

#     #json object
#     line = json.loads(row) 
#     print(line["Muhammed MacIntyre"]) 
#     # print(line)

Streaming output truncated to the last 5000 lines.
###################################################################################################################
Harold Dahlen
{"Eldon Base for stackable storage shelf, platinum":"Fellowes 17-key keypad for PS/2 interface","Muhammed MacIntyre":"Clytie Kelty","3":"4645","-213.25":"-45.07","38.94":"30.73","35":"4","Nunavut":"Alberta","Storage & Organization":"Computer Peripherals","0.8":"0.75"}
###################################################################################################################
Clytie Kelty
{"Eldon Base for stackable storage shelf, platinum":"Acco 6 Outlet Guardian Premium Surge Suppressor","Muhammed MacIntyre":"Clytie Kelty","3":"4645","-213.25":"-8.53","38.94":"14.56","35":"3.5","Nunavut":"Alberta","Storage & Organization":"Appliances","0.8":"0.58"}
###################################################################################################################
Clytie Kelty
{"Eldon Base for stackable

Convert CSV to Parquet

In [33]:
start_time = datetime.now()
print ("Reading CSV file started at : ",start_time)
df = spark.read.load('/content/drive/MyDrive/PBDM_Dataset/Sample-Spreadsheet-500000-rows.csv', format='csv', inferSchema=False, header=False)
df.write.parquet("/content/drive/MyDrive/PBDM_Dataset/parquet_output/")
end_time = datetime.now()
print ("Writing as Parquet completed at: ",end_time)
print ("Total Duration in Converting CSV to Parquet: ",end_time-start_time)

Reading CSV file started at :  2022-11-16 03:26:53.390479
Writing as Parquet completed at:  2022-11-16 03:26:55.352221
Total Duration in Converting CSV to Parquet:  0:00:01.961742


In [36]:
df_parquet = spark.read.parquet("/content/drive/MyDrive/PBDM_Dataset/parquet_output")
df_parquet.count()

59507

Convert Parquet to Parquet

In [34]:
start_time = datetime.now()
print ("Reading Parquet file started at : ",start_time)
df = spark.read.parquet("/content/drive/MyDrive/PBDM_Dataset/parquet_output")
df.write.mode("overwrite").parquet("/content/drive/MyDrive/PBDM_Dataset/parquet_to_parquet_output")
end_time = datetime.now()
print ("Writing as Parquet completed at: ",end_time)
print ("Total Duration in Converting Parquet to Parquet: ",end_time-start_time)

Reading Parquet file started at :  2022-11-16 03:30:40.972232
Writing as Parquet completed at:  2022-11-16 03:30:42.448722
Total Duration in Converting Parquet to Parquet:  0:00:01.476490


Aggregations on CSV file

In [41]:
df = df = spark.read.load('/content/drive/MyDrive/PBDM_Dataset/Sample-Spreadsheet-500000-rows.csv', format='csv', inferSchema=False, header=False)
start_time = datetime.now()
#print Group and count
df.groupBy('_c1').count().show()
#print avg
df.select(mean('_c2')).show()
#print min
df.select(min('_c2')).show()
#print max
df.select(max('_c2')).show()
#print count of unique
df.select(countDistinct('_c2')).show()
end_time = datetime.now()
print ("Total Duration for the above aggregations on csv file: ",end_time-start_time)

+-----------------+-----+
|              _c1|count|
+-----------------+-----+
|     Jesus Ocampo|   30|
|      Jim Mitchum|   60|
|         Joy Bell|   66|
|    Hilary Holden|   66|
|  Patrick O'Brill|   54|
|   Parhena Norris|   42|
|     Ruben Ausman|   48|
| Michelle Ellison|  108|
|     Ben Peterman|   42|
|         Jay Fine|  116|
|  Ted Butterfield|   12|
|    Bill Overfelt|   60|
|        Denny Joy|   42|
|      Joseph Holt|   18|
|     Melanie Page|   72|
|     Xylona Price|   84|
|       Carl Weiss|  126|
|Brooke Gillingham|  108|
|   Darren Koutras|   42|
|     Craig Yedwab|   42|
+-----------------+-----+
only showing top 20 rows

+------------------+
|          avg(_c2)|
+------------------+
|30586.989853104984|
+------------------+

+---------+
| min(_c2)|
+---------+
| 10/Pack"|
+---------+

+--------------+
|      max(_c2)|
+--------------+
|Yoseph Carroll|
+--------------+

+-------------------+
|count(DISTINCT _c2)|
+-------------------+
|               5585|
+--------

Aggragations on Parquet File

In [42]:
df = spark.read.parquet("/content/drive/MyDrive/PBDM_Dataset/parquet_output")
start_time = datetime.now()
#print Group and count
df.groupBy('_c1').count().show()
#print avg
df.select(mean('_c2')).show()
#print min
df.select(min('_c2')).show()
#print max
df.select(max('_c2')).show()
#print count of unique
df.select(countDistinct('_c2')).show()
end_time = datetime.now()
print ("Total Duration for the above aggregations on parquet file: ",end_time-start_time)

+-----------------+-----+
|              _c1|count|
+-----------------+-----+
|     Jesus Ocampo|   30|
|      Jim Mitchum|   60|
|         Joy Bell|   66|
|    Hilary Holden|   66|
|  Patrick O'Brill|   54|
|   Parhena Norris|   42|
|     Ruben Ausman|   48|
| Michelle Ellison|  108|
|     Ben Peterman|   42|
|         Jay Fine|  116|
|  Ted Butterfield|   12|
|    Bill Overfelt|   60|
|        Denny Joy|   42|
|      Joseph Holt|   18|
|     Melanie Page|   72|
|     Xylona Price|   84|
|       Carl Weiss|  126|
|Brooke Gillingham|  108|
|   Darren Koutras|   42|
|     Craig Yedwab|   42|
+-----------------+-----+
only showing top 20 rows

+------------------+
|          avg(_c2)|
+------------------+
|30586.989853104984|
+------------------+

+---------+
| min(_c2)|
+---------+
| 10/Pack"|
+---------+

+--------------+
|      max(_c2)|
+--------------+
|Yoseph Carroll|
+--------------+

+-------------------+
|count(DISTINCT _c2)|
+-------------------+
|               5585|
+--------